In [ ]:
import os
import json
import requests
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, ToolMessage

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client via LangChain
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

### Find Country based on Location/Position

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
import requests
import os
import re

# Initialize model for coordinate processing
model_low_temp = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0.1
)

# Define the state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_input: str
    latitude: float
    longitude: float
    country_name: str
    full_instruction: str

# Define input/output
class Input(TypedDict):
    user_input: str

class Output(TypedDict):
    country_name: str
    coordinates: dict
    original_instruction: str

# System prompt for coordinate extraction from complex text
coordinate_extraction_prompt = SystemMessage(
    """You are an expert at extracting coordinates from complex instructions.
    
    Look for coordinate patterns in the text such as:
    - [latitude, longitude] in brackets
    - (latitude, longitude) in parentheses  
    - "at latitude, longitude"
    - "positioned at lat, lon"
    - Any decimal numbers that could be coordinates
    
    Rules:
    - Latitude: -90 to 90 degrees
    - Longitude: -180 to 180 degrees
    - Return ONLY the numbers in format: "latitude,longitude"
    - If no valid coordinates found, return "INVALID"
    
    Examples:
    "positioned at [34.0522, -118.2437]" → "34.0522,-118.2437"
    "Create drone pad at (40.7128, -74.0060)" → "40.7128,-74.0060"
    "Setup at coordinates 48.8566, 2.3522 for mission" → "48.8566,2.3522"
    """
)

# Extract coordinates from complex instruction
def extract_coordinates(state: State) -> State:
    user_message = HumanMessage(state["user_input"])
    messages = [coordinate_extraction_prompt, user_message]
    
    res = model_low_temp.invoke(messages)
    coordinate_result = res.content.strip()
    
    # Also try regex pattern matching as backup
    if coordinate_result == "INVALID":
        # Look for patterns like [lat, lon], (lat, lon), or lat, lon
        patterns = [
            r'\[([+-]?\d+\.?\d*),\s*([+-]?\d+\.?\d*)\]',  # [34.0522, -118.2437]
            r'\(([+-]?\d+\.?\d*),\s*([+-]?\d+\.?\d*)\)',  # (34.0522, -118.2437)
            r'positioned at ([+-]?\d+\.?\d*),\s*([+-]?\d+\.?\d*)',  # positioned at 34.0522, -118.2437
            r'at ([+-]?\d+\.?\d*),\s*([+-]?\d+\.?\d*)',  # at 34.0522, -118.2437
            r'coordinates ([+-]?\d+\.?\d*),\s*([+-]?\d+\.?\d*)',  # coordinates 34.0522, -118.2437
        ]
        
        for pattern in patterns:
            match = re.search(pattern, state["user_input"], re.IGNORECASE)
            if match:
                lat, lon = match.groups()
                coordinate_result = f"{lat},{lon}"
                break
    
    if coordinate_result == "INVALID":
        return {
            "latitude": None,
            "longitude": None,
            "full_instruction": state["user_input"],
            "messages": [user_message, res],
        }
    
    try:
        lat_str, lon_str = coordinate_result.split(',')
        latitude = float(lat_str.strip())
        longitude = float(lon_str.strip())
        
        # Validate coordinate ranges
        if not (-90 <= latitude <= 90) or not (-180 <= longitude <= 180):
            raise ValueError("Invalid coordinate range")
            
        return {
            "latitude": latitude,
            "longitude": longitude,
            "full_instruction": state["user_input"],
            "messages": [user_message, res],
        }
    except:
        return {
            "latitude": None,
            "longitude": None,
            "full_instruction": state["user_input"],
            "messages": [user_message, res],
        }

# Find country name from coordinates
def find_country(state: State) -> State:
    if state["latitude"] is None or state["longitude"] is None:
        return {
            "country_name": "No valid coordinates found",
            "messages": state["messages"],
        }
    
    try:
        url = "https://nominatim.openstreetmap.org/reverse"
        params = {
            "lat": state["latitude"],
            "lon": state["longitude"],
            "format": "json",
            "addressdetails": 1,
            "zoom": 3,
            "accept-language": "en"  # Force English language response
        }
        
        headers = {"User-Agent": "LangGraph-Country-Finder/1.0"}
        response = requests.get(url, params=params, headers=headers, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            country_name = data.get("address", {}).get("country", "Unknown")
            return {
                "country_name": country_name,
                "messages": state["messages"],
            }
        else:
            return {
                "country_name": "Geocoding service unavailable",
                "messages": state["messages"],
            }
            
    except Exception as e:
        return {
            "country_name": f"Error: {str(e)}",
            "messages": state["messages"],
        }

# Build the graph
builder = StateGraph(State, input_schema=Input, output_schema=Output)
builder.add_node("extract_coordinates", extract_coordinates)
builder.add_node("find_country", find_country)

builder.add_edge(START, "extract_coordinates")
builder.add_edge("extract_coordinates", "find_country")
builder.add_edge("find_country", END)

graph = builder.compile()

# Interactive usage - ask user for input
if __name__ == "__main__":
    # print("=== Coordinate to Country Finder ===")
    # print("Enter instructions containing coordinates (or 'quit' to exit)")
    # print("\nExample: Create a polygonal TLOF positioned at [34.0522, -118.2437]")
    
    while True:
        # print("\n" + "-" * 50)
        user_instruction = input("Enter your positions: ").strip()
        
        if user_instruction.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
            
        if not user_instruction:
            print("Please enter a valid positions.")
            continue
        
        # print("\nProcessing...")
        
        try:
            result = graph.invoke({"user_input": user_instruction})
            
            print(f"Country: {result['country_name']}")
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            print("Please try again with a different positions.")

Enter your instruction:  52.2870, 104.2810


Country: Russia


Enter your instruction:  92.2870, 104.2810


Country: No valid coordinates found


Enter your instruction:  13.08784, 80.27847


Country: India


KeyboardInterrupt: Interrupted by user

### LangChain RAG using Azure embedding

In [ ]:
import os
from dotenv import load_dotenv

# from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

load_dotenv()  # Ensure this is called first

# --- Azure model setup ---
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),  # e.g., "gpt-4o" or "gpt-35-turbo"
    temperature=0
)


# Single file load and process PDF
'''
pdf_path = r"C:\_IESEG\_Internship\_Lyneports\_LyneportProject\_regulations\EASA_USA\Copy of EASA Prototype Technical Design Specifications for Vertiports_March2022.pdf"
loader = PyPDFLoader(pdf_path)
'''

# Multiple file load from directory and process PDF
loader = DirectoryLoader(
    "./_regulations/FAA_USA",
    glob="**/*.pdf",  # recursive search
    loader_cls=PyPDFLoader
)

# Load all matching PDF files from the _regulations folder into a list of Document objects
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Embeddings (Azure uses OpenAI API, so this works with correct API key)
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_EMBEDDING_MODEL_NAME"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),  # Correct key now
    #openai_api_key=os.getenv("AZURE_EMBEDDING_API_KEY"),
    openai_api_version="2024-02-01",  # or whatever version you're using
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever
)

# Example query
# Summarize the design requirements for vertiport landing areas.
# What is VTOL-capable aircraft?
# What is MTOW?
# TLOF Guidance in 5 lines
# Takeoff and Landing Area Minimum Dimensions for TLOF

query = "Takeoff and Landing Area Minimum Dimensions for TLOF"

result = qa_chain.run(query)

print(result)


For non-powered-lift scenarios, the minimum dimension for the TLOF (Touchdown and Lift-Off Area) is 1.88 D-p. For powered-lift scenarios, the minimum dimension for the TLOF is 1 D-p.


### RAG using Azure Search

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import AzureSearch

load_dotenv()

# Azure model setup
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

# Create embeddings object (must match the one used to create your index)
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_EMBEDDING_MODEL_NAME"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),
   # openai_api_key=os.getenv("AZURE_EMBEDDING_API_KEY"),
    openai_api_version="2024-02-01"
)

# Use Azure Cognitive Search vectorstore
vectorstore = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT"),
    #azure_search_key=os.getenv("AZURE_SEARCH_ADMIN_KEY"),
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
    embedding_function=embeddings.embed_query
)

retriever = vectorstore.as_retriever()

# --- RetrievalQA chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# --- Run a query ---
query = "Takeoff and Landing Area Minimum Dimensions for TLOF"
result = qa_chain.run(query)

print(result)


HttpResponseError: (InvalidRequestParameter) Unknown field 'content_vector' in vector field list.
Code: InvalidRequestParameter
Message: Unknown field 'content_vector' in vector field list.
Exception Details:	(UnknownField) Unknown field 'content_vector' in vector field list.
	Code: UnknownField
	Message: Unknown field 'content_vector' in vector field list.

In [ ]:
import requests
import os

# Azure Search config
endpoint = "https://reportgeneration.search.windows.net"
index_name = "lyneportsindex"
api_version = "2021-04-30-Preview"
#api_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  # store your key securely

# Search query
search_text = "Minimum Dimension for TLOF"

# Call Azure Search REST API
url = f"{endpoint}/indexes/{index_name}/docs/search?api-version={api_version}"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}
payload = {
    "search": search_text,   # simple keyword search
    "top": 5                 # number of results
}

response = requests.post(url, headers=headers, json=payload)
results = response.json()

# Print results
for doc in results.get("value", []):
    print(f"Title: {doc.get('title')}")
    print(f"Content: {doc.get('content')[:200]}...\n")

Title: FAA AC_150_5390_2D_Heliports (2023).pdf
Content: headings (N, S, E, W) and four intercardinal headings (NE, SE, SW, and NW). 

 Use a minimum dimension of a 6-foot (1.8 m) outer diameter and a 4-foot (1.2 m) inner diameter 
for the compass circle. 
...

Title: FAA AC_150_5390_2D_Heliports (2023).pdf
Content: illustrates this alternative marking. 

4.3.3 Private-use (PVT) Identification Marking. 
An optional “PVT” marking may be used to indicate a private-use heliport. Text height 
is a minimum of 3 feet (...

Title: FAA AC_150_5390_2D_Heliports (2023).pdf
Content: than a circular shape, but a circular TLOF may be more recognizable in an urban 
environment. Increasing the TLOF and the size of the LBA centered on the TLOF may 
provide enhanced safety and operatio...

Title: FAA eb-105-vertiports.pdf
Content: design until there is adequate research on these emerging aircraft to develop a 

performance-based vertiport design AC.  Accordingly, the aircraft features and 

performance 